In [1]:
% setup MRI-education-resources path and requirements
cd ../
startup

loading image
loading signal


# Image Reconstruction

Image reconstruction in MRI means converting the raw signal collected in k-space into images.  For a fully-sampled acquisition using Cartesian sampling (e.g. on a grid), this can be done with a discrete Fourier Transform.  In anticipation of advanced reconstruction methods, image reconstruction for MRI is formulated as a linear system.  The technique of Partial Fourier imaging is also introduced.

## Learning Goals

1. Describe how images are formed
    * Describe how MRI raw data is reconstructed into an image
1. Manipulate MRI sequence parameters to improve performance
    * Describe how the Partial Fourier method can be used
1. Manipulate and analyze MRI data
    * Reconstruct an image from raw data


## Sorting the MRI Data into k-space

Recall that the MRI signal is proportional to the Fourier Transform of the net magnetization of our object, evaluated at the k-space location defined by the gradients:

$$ s_i(t) = \mathcal{F} \{ M_{XY}(\vec{r},0) \} |_{\vec{k} = \vec{k}_i(t)} $$

Note that this includes a dimension for the readout, $t$, as well as different readouts for each TR, denoted by the subscript for the $i^\mathrm{th}$ TR.

The data acquired during the readout will be sorted into data based on the k-space trajectory.  The following example shows the Cartesian k-space trajectory, which acquired data in a raster fashion in k-space:

![Cartesian k-space trajectory and pulse sequence](images/frequency_phase_encoding-full-image_phase.gif)

From this knowledge, the MR signal over time is stored in a data matrix with known k-space locations to create $M(\vec{k})$.

## Fourier Transform Reconstruction

Once the data has been sorted into the corresponding lines in k-space, an inverse Fourier Transform is applied to reconstruct an image of the transverse magnetization

$$\mathcal{F}^{-1}\{ M(\vec{k} )\} = m(\vec{r})$$

For typical Cartesian sampling, this can be done very simply with the 2D or 3D Fast Fourier Transform (FFT) algorithm

The following animation illustrates typical k-space data as it would be acquired for different k-space lines (left) and the resulting image as more and more lines of k-space are accumulated

![Image formation from sequential Cartesian k-space data](images/image_formation_sequential.gif)

We can also acquire our k-space lines in a "center-out" or random ordering, shown below

![Image formation from center-out Cartesian k-space data](images/image_formation_centerout_lines.gif)
![Image formation from random Cartesian k-space data](images/image_formation_random_lines.gif)

## Partial Fourier

Partial Fourier methods, also known as fractional NEX, are a way to sample a reduced amount of k-space while still reconstructing an image.

These methods note one of the Fourier Transform properties which states that the positive k-space data equals the complex conjugate of the negative k-space data when the object is real-valued:

$$M(\vec{k}) = M^*(-\vec{k}), \ \mathrm{if} \ \mathcal{Im}(m(\vec{r})) = 0$$

Our object is the transverse magnetization, so if our transverse magnetization is all aligned along $M_X$ then our object is real-valued and we can apply this relationship.

$$m(\vec{r}) = M_{XY}(\vec{r},0) = M_{X}(\vec{r},0) + i M_{Y}(\vec{r},0) $$

Under ideal conditions, the RF pulse can excite the net magnetization just into $M_X$, meaning this requirement can be satisfied.    Taken together, this means only half of the k-space data (e.g positive data $\vec{k} \geq 0$) can be acquired and it can be used to fill in the negative k-space data.  This means 2 times faster scanning!


In practice, there are sources of phase that result in violation of the real-valued object condition.  First, there can be a global phase offset (e.g. transverse magnetization aligned along $M_Y$), but this is not a problem since a global phase correction can be applied to correct for this.  More problematic are other sources of phase, which come from the RF coil profiles, off-resonance, and chemical shift.  

However, this additional phase is typically contained in the low spatial frequencies, in other words they can be measured by our central k-space values.  Thus, practical Partial Fourier methods acquire slightly more than half of k-space, and then use specialized algorithms such as projection onto convex sets (POCS) or homodyne detection reconstructions that include correction of the low spatial frequency phase.

In [ ]:
% Partial Fourier examples

dataname = 'Data/se_t1_sag_data';
load(dataname)
kdata = data;
S = size(kdata);

im_original = ifft2c(kdata);

subplot(2,1,1)
imagesc(real(im_original))
title('Real(image)')
colormap(gray), axis equal tight off
subplot(2,1,2)
imagesc(imag(im_original))
title('Imag(image)')
colormap(gray), axis equal tight off


% is this image suitable for Partial Fourier reconstruction?